In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
pd.options.display.float_format = '{:.5f}'.format

raw 데이터인 user_spec과 loan_result를 병합한 데이터를 train과 test로 나눈 csv 호출

In [4]:
train = pd.read_csv("/content/drive/MyDrive/빅콘테스트/bigcon_train_origin.csv", index_col = 0)

In [5]:
test = pd.read_csv("/content/drive/MyDrive/빅콘테스트/bigcon_test_origin.csv", index_col = 0)

In [6]:
log_data = pd.read_csv("/content/drive/MyDrive/빅콘테스트/빅콘테스트_데이터셋/log_data.csv")

## test data application_id로 정렬

제출용 평가데이터 파일이 application_id를 기준으로 정렬되어 있으므로 test data 정렬

In [7]:
test = test.sort_values(['application_id'])

In [8]:
test.reset_index(drop=True, inplace =True)

## bank_rank 파생변수 생성

is_applied가 1인 데이터 중 bank_id의 개수가 제일 많은 순서대로 그룹화하여 순위를 결정한 뒤 bank_rank 변수 생성

In [9]:
train['bank_rank'] = 0
test['bank_rank'] = 0

In [10]:
bank_list = [59, 29, 49, 38]

for i in bank_list:
    idx = train[train['bank_id'].isin([i])]['bank_id'].index
    idx2 = test[test['bank_id'].isin([i])]['bank_id'].index
    train.loc[idx,'bank_rank'] = 6
    test.loc[idx2,'bank_rank'] = 6

In [11]:
bank_list = [2, 35, 27, 1, 36]

for i in bank_list:
    idx = train[train['bank_id'].isin([i])]['bank_id'].index
    idx2 = test[test['bank_id'].isin([i])]['bank_id'].index
    train.loc[idx,'bank_rank'] = 5
    test.loc[idx2,'bank_rank'] = 5

In [12]:
bank_list = [4, 33, 6, 40, 8, 63, 50, 13, 19, 21, 23, 30, 47, 52]

for i in bank_list:
    idx = train[train['bank_id'].isin([i])]['bank_id'].index
    idx2 = test[test['bank_id'].isin([i])]['bank_id'].index
    train.loc[idx,'bank_rank'] = 4
    test.loc[idx2,'bank_rank'] = 4

In [13]:
bank_list = [58, 10, 54, 22, 32, 17, 5, 31, 43, 45, 7, 37]

for i in bank_list:
    idx = train[train['bank_id'].isin([i])]['bank_id'].index
    idx2 = test[test['bank_id'].isin([i])]['bank_id'].index
    train.loc[idx,'bank_rank'] = 3
    test.loc[idx2,'bank_rank'] = 3

In [14]:
bank_list = [14, 56, 41, 42, 15, 24, 34,57, 44, 60, 11, 25, 55, 2, 64, 18, 3, 26, 20, 46, 51, 12]

for i in bank_list:
    idx = train[train['bank_id'].isin([i])]['bank_id'].index
    idx2 = test[test['bank_id'].isin([i])]['bank_id'].index
    train.loc[idx,'bank_rank'] = 2
    test.loc[idx2,'bank_rank'] = 2

In [15]:
bank_list = [61, 39, 9, 28, 16]

for i in bank_list:
    idx = train[train['bank_id'].isin([i])]['bank_id'].index
    idx2 = test[test['bank_id'].isin([i])]['bank_id'].index
    train.loc[idx,'bank_rank'] = 1
    test.loc[idx2,'bank_rank'] = 1

## train_event 생성

log_data의 event칼럼의 행동명들의 횟수를 칼럼으로 변환

In [16]:
df = log_data[['user_id','event']].value_counts().to_frame()

In [17]:
df.reset_index(inplace=True)

In [18]:
df.rename(columns = {0: 'count'}, inplace=True)

In [19]:
event = ['SignUp','OpenApp','Login','ViewLoanApplyIntro','StartLoanApply','CompleteIDCertification',
         'EndLoanApply','UseLoanManage','UsePrepayCalc','UseDSRCalc','GetCreditInfo']

for i in event:
    df_new = df[df['event']==i]
    df_new.drop(['event'], axis= 1, inplace=True)
    train = pd.merge(train,df_new, how = 'outer') 
    train.rename(columns = {"count": i}, inplace = True)

In [20]:
for i in event:
    df_new = df[df['event']==i]
    df_new.drop(['event'], axis= 1, inplace=True)
    test = pd.merge(test,df_new, how = 'outer') 
    test.rename(columns = {"count": i}, inplace = True)

In [21]:
train.dropna(axis=0, subset=['application_id'], inplace= True)

In [22]:
test.dropna(axis=0, subset=['application_id'], inplace= True)

In [23]:
for i in event:
    train[i].fillna(0, inplace=True)
    test[i].fillna(0, inplace=True)

## 불필요한 칼럼 삭제

모델링에 불필요하다고 판단되는 칼럼 drop  
company_enter_month칼럼은 데이터의 신뢰성이 낮아 제거

In [24]:
train.drop(['SignUp','OpenApp','ViewLoanApplyIntro','StartLoanApply','CompleteIDCertification',
            'insert_time', 'loanapply_insert_time','company_enter_month','gender','personal_rehabilitation_complete_yn',
            'personal_rehabilitation_yn'], axis= 1, inplace=True)

In [25]:
test.drop(['SignUp','OpenApp','ViewLoanApplyIntro','StartLoanApply','CompleteIDCertification',
            'insert_time', 'loanapply_insert_time','company_enter_month','gender','personal_rehabilitation_complete_yn',
            'personal_rehabilitation_yn'], axis= 1, inplace=True)

## 이상치 처리

loan_limit칼럼의 비정상적인 값 제거

In [26]:
# 승인 한도가 99억 이상인 값 제거
idx = train[train['loan_limit']>9900000000].index

In [27]:
train.drop(idx, axis=0, inplace=True)

In [28]:
train.reset_index(drop=True, inplace =True)

IQR 기준 이상치 제거

In [29]:
def get_outlier(df=None, column=None, weight=1.5):
  quantile_25 = np.percentile(df[column].values, 25)
  quantile_75 = np.percentile(df[column].values, 75)

  IQR = quantile_75 - quantile_25
  IQR_weight = IQR*weight
  
  lowest = quantile_25 - IQR_weight
  highest = quantile_75 + IQR_weight
  
  outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
  return outlier_idx

# 함수 사용해서 이상치 값 삭제
outlier_list = ['yearly_income','desired_amount','loan_limit','Login','EndLoanApply','UseLoanManage','UsePrepayCalc','UseDSRCalc','GetCreditInfo']

for i in outlier_list:
    oulier_idx = get_outlier(df=train, column=i, weight=1.5)
    train.drop(oulier_idx, axis=0, inplace=True)

In [30]:
train.reset_index(drop=True, inplace =True)

In [31]:
test.reset_index(drop=True, inplace =True)

## 생년월일 -> 나이로 변환

현재 2022년을 기준으로 나이 변환

In [32]:
train['age'] = 2022 - train['birth_year'] + 1
train.drop('birth_year', axis = 1, inplace = True)

In [33]:
test['age'] = 2022 - test['birth_year'] + 1
test.drop('birth_year', axis = 1, inplace = True)

## admission_count 파생변수 생성

특정 application_id의 개수가 곧 product_id의 개수(승인된 상품 개수)이므로   
application_id 개수를 기준으로 승인된 상품 개수를 센 후 admission_count 변수 생성

In [34]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [35]:
train_sort = train.sort_values('application_id')
test_sort = test.sort_values('application_id')

In [36]:
admission_count_train = pd.DataFrame(train['application_id'].value_counts().sort_index())
admission_count_test = pd.DataFrame(test['application_id'].value_counts().sort_index())

In [37]:
admission_count_train.columns = ['admission_count']
admission_count_test.columns = ['admission_count']

In [38]:
admission_count_train.index.name = 'application_id'
admission_count_test.index.name = 'application_id'

In [39]:
admission_count_train = admission_count_train.reset_index()
admission_count_test = admission_count_test.reset_index()

In [40]:
train = pd.merge(train_sort, admission_count_train)
test = pd.merge(test_sort, admission_count_test)

## 인코딩

범주형 변수들 LabelEncoding

In [41]:
features = ['income_type', 'employment_type', 'houseown_type', 'purpose']

for o_col in features:
    encoder = LabelEncoder()
    encoder.fit(train[o_col])
    train_encoded = encoder.transform(train[o_col])
    test_encoded = encoder.transform(test[o_col])

    train[o_col] = train_encoded
    test[o_col] = test_encoded

## 스케일링

In [42]:
# 범위가 큰 값 로그변환
# 이를 통해 이상치 문제 또한 완화할 수 있음
scale_columns = ['credit_score','yearly_income','desired_amount','existing_loan_amt','loan_limit',
                 'Login','EndLoanApply','UseLoanManage','UsePrepayCalc','UseDSRCalc','GetCreditInfo']

for i in scale_columns:
    train[i] = np.log1p(train[i])

In [43]:
for i in scale_columns:
    test[i] = np.log1p(test[i])

## 결측치 처리

In [44]:
# 결측치 확인 함수
def check_missing_col(dataframe):
    missing_col = []
    counted_missing_col = 0
    for i,col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col +=1
            print('결측치가 있는 칼럼은: %s 입니다.'%col)
            print('해당 칼럼에 총 %s개의 결측치가 존재합니다. '%missing_values)
            missing_col.append([col, dataframe[col].dtype])
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다.')
    return missing_col

In [45]:
missing_col = check_missing_col(train)

결측치가 있는 칼럼은: credit_score 입니다.
해당 칼럼에 총 1023104개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: existing_loan_cnt 입니다.
해당 칼럼에 총 1707913개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: existing_loan_amt 입니다.
해당 칼럼에 총 2224116개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: loan_limit 입니다.
해당 칼럼에 총 3793개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: loan_rate 입니다.
해당 칼럼에 총 3793개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: age 입니다.
해당 칼럼에 총 70142개의 결측치가 존재합니다. 


In [46]:
missing_col = check_missing_col(test)

결측치가 있는 칼럼은: credit_score 입니다.
해당 칼럼에 총 265464개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: yearly_income 입니다.
해당 칼럼에 총 6개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: existing_loan_cnt 입니다.
해당 칼럼에 총 541898개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: existing_loan_amt 입니다.
해당 칼럼에 총 846023개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: loan_limit 입니다.
해당 칼럼에 총 1757개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: loan_rate 입니다.
해당 칼럼에 총 1757개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: age 입니다.
해당 칼럼에 총 36470개의 결측치가 존재합니다. 


existing_loan_cnt : 0이 없기 때문에 기대출이 없다고 판단하여 0으로 채워줌


In [47]:
train['existing_loan_cnt'].fillna(0, inplace= True)

In [48]:
test['existing_loan_cnt'].fillna(0, inplace= True)

existing_loan_amt : 기대출수가 결측치이면 기대출금액도 결측치이기 때문에 0으로 채움, 나머지 900292개의 결측치는 IterativeImputer를 이용해 채움


In [49]:
idx2 = train[train['existing_loan_cnt']==0].index
train.loc[idx2,'existing_loan_amt'] = 0

In [50]:
idx2 = test[test['existing_loan_cnt']==0].index
test.loc[idx2,'existing_loan_amt'] = 0

loan_rate, loan_limit 컬럼에 결측치(nan)가 있는 경우는 금융사에서 값을 보내주지 않은 경우로,  해당 경우는 채점에서 제외할 예정이니 제거



In [51]:
train.dropna(subset=['loan_limit'], axis=0, inplace=True)
train.dropna(subset=['loan_rate'], axis=0, inplace=True)

In [52]:
test.dropna(subset=['loan_limit'], axis=0, inplace=True)
test.dropna(subset=['loan_rate'], axis=0, inplace=True)

In [53]:
train.reset_index(drop=True, inplace =True)

In [54]:
test.reset_index(drop=True, inplace =True)

결측치 채운 후 남은 결측치 확인

In [55]:
missing_col = check_missing_col(train)

결측치가 있는 칼럼은: credit_score 입니다.
해당 칼럼에 총 1022646개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: existing_loan_amt 입니다.
해당 칼럼에 총 516004개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: age 입니다.
해당 칼럼에 총 70103개의 결측치가 존재합니다. 


In [56]:
missing_col = check_missing_col(test)

결측치가 있는 칼럼은: credit_score 입니다.
해당 칼럼에 총 265321개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: yearly_income 입니다.
해당 칼럼에 총 6개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: existing_loan_amt 입니다.
해당 칼럼에 총 304033개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: age 입니다.
해당 칼럼에 총 36453개의 결측치가 존재합니다. 


## aplication_id, product_id 중복 제거

제출용 평가데이터에서 application_id와 product_id가 중복된 값을 제거하였으므로 동일하게 test데이터에서 제거

In [57]:
test[test.duplicated(['application_id','product_id'],keep=False)][['application_id','product_id']].sort_values(['application_id'])

,application_id,product_id
1824,1561.00000,51.00000
1827,1561.00000,51.00000
2697,2407.00000,51.00000
2698,2407.00000,51.00000
27917,19876.00000,267.00000
...,...,...
3234267,2153509.00000,51.00000
3245577,2160853.00000,65.00000
3245580,2160853.00000,65.00000
3249787,2163982.00000,168.00000


In [58]:
test = test.drop_duplicates(['application_id','product_id'], keep=False)

## IterativeImputer를 이용해 결측치 채우기

credit_score 결측치 채우기

In [59]:
# credit score랑 상관계수가 0.2 이상인 변수들 선택
train_miss = train[['desired_amount', 'existing_loan_cnt', 'loan_limit', 'loan_rate', 'admission_count', 'credit_score']]
test_miss = test[['desired_amount', 'existing_loan_cnt', 'loan_limit', 'loan_rate', 'admission_count', 'credit_score']]

In [60]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(random_state=42)

train_trans = imputer.fit_transform(train_miss)
train_miss = pd.DataFrame(train_trans, columns=train_miss.columns)

In [61]:
imputer = IterativeImputer(random_state=42)

test_miss_copy = test_miss.copy()
imputer.fit(train_miss)   
test_trans = imputer.transform(test_miss)
test_miss = pd.DataFrame(test_trans, columns=test_miss_copy.columns)

In [62]:
train['credit_score']=train_miss['credit_score']
test['credit_score']=test_miss['credit_score']

train의 나머지 결측치 행 제거

In [64]:
train.dropna(axis = 0, inplace = True)

In [65]:
train.reset_index(drop=True, inplace=True)

test의 나머지 결측치 imputer를 이용해 채움

In [66]:
missing_col = check_missing_col(test)

결측치가 있는 칼럼은: credit_score 입니다.
해당 칼럼에 총 288개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: yearly_income 입니다.
해당 칼럼에 총 6개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: existing_loan_amt 입니다.
해당 칼럼에 총 304019개의 결측치가 존재합니다. 
결측치가 있는 칼럼은: age 입니다.
해당 칼럼에 총 36453개의 결측치가 존재합니다. 


In [67]:
imputer = IterativeImputer(random_state=42)

test_columns = test.columns
imputer.fit(train.drop(['is_applied'], axis=1))  
test = pd.DataFrame(imputer.transform(test), columns= test_columns)

## credit_loan_rate 파생변수 생성

여신금융협회에서 신용점수 별 대출금리의 평균을 구한 뒤 각 신용점수 구간에 해당하는 loan_rate와 비교하여 loan_rate가 더 높을 경우 0, 낮을 경우 1의 값을 할당

In [68]:
train['credit_loan_rate']= 1
test['credit_loan_rate']= 1

In [69]:
score300 = (16.99+18.45)/2
score400 = 19.9
score500 = (19.18+19.9+19.9)/3
score600 = (19.9+14.37+18.27+15.78+17.9+17.57+14.65+19.13+10.9+18.5+16.86+18.79)/12
score700 = (19.59+14.29+16.71+14.89+14.53+16.81+14.14+18.74+12.25+18.25+16.12+16.12)/12
score800 = (16.18+11.71+13.03+15.57+13.59+13.02+15.58+12.82+17.53+11.65+17.55+14.68+14.26)/13
score900 = (13.90+11.87+10.7+14.54+12.69+12.75+13.51+11.83+15.75+10.71+15.53+12.89+12.75)/13
score1000 = (12.62+11.26+9.28+13.05+12.22+10.45+11.29+10.60+12.21+10.25+12.92+11.43+11.50)/13

In [70]:
cond1 = (train['credit_score'] <= 300) & (train['loan_rate'] > score300)
cond2 = (train['credit_score'] > 300) & (train['credit_score'] <= 400) & (train['loan_rate'] > score400)
cond3 = (train['credit_score'] > 400) & (train['credit_score'] <= 500) & (train['loan_rate'] > score500)
cond4 = (train['credit_score'] > 500) & (train['credit_score'] <= 600) & (train['loan_rate'] > score600)
cond5 = (train['credit_score'] > 600) & (train['credit_score'] <= 700) & (train['loan_rate'] > score700)
cond6 = (train['credit_score'] > 700) & (train['credit_score'] <= 800) & (train['loan_rate'] > score800)
cond7 = (train['credit_score'] > 800) & (train['credit_score'] <= 900) & (train['loan_rate'] > score900)
cond8 = (train['credit_score'] > 900) & (train['loan_rate'] > score1000)

In [71]:
train['credit_loan_rate'] = np.where(cond1,0,np.where(cond2,0,np.where(cond3,0,np.where(cond4,0,np.where(cond5,0,np.where(cond6,0,np.where(cond7,0,np.where(cond8,0,1))))))))

In [72]:
cond1 = (test['credit_score'] <= 300) & (test['loan_rate'] > score300)
cond2 = (test['credit_score'] > 300) & (test['credit_score'] <= 400) & (test['loan_rate'] > score400)
cond3 = (test['credit_score'] > 400) & (test['credit_score'] <= 500) & (test['loan_rate'] > score500)
cond4 = (test['credit_score'] > 500) & (test['credit_score'] <= 600) & (test['loan_rate'] > score600)
cond5 = (test['credit_score'] > 600) & (test['credit_score'] <= 700) & (test['loan_rate'] > score700)
cond6 = (test['credit_score'] > 700) & (test['credit_score'] <= 800) & (test['loan_rate'] > score800)
cond7 = (test['credit_score'] > 800) & (test['credit_score'] <= 900) & (test['loan_rate'] > score900)
cond8 = (test['credit_score'] > 900) & (test['loan_rate'] > score1000)

In [73]:
test['credit_loan_rate'] = np.where(cond1,0,np.where(cond2,0,np.where(cond3,0,np.where(cond4,0,np.where(cond5,0,np.where(cond6,0,np.where(cond7,0,np.where(cond8,0,1))))))))

## age를 범주화하여 age_level 생성

모든 기준은 대한민국 평균 연령로 결정  
1) 평균 취업 연령 : 28세  
2) 평균 결혼 및 출산 연령 : 32세  
3) 평균 내 집 마련 연령 : 40세  
4) 평균 은퇴 연령 : 60세   
를 이용해 age를 범주화하여 age_level 생성

In [74]:
condition1 = (train['age'] < 28)
condition2 = (train['age'] >= 28) & (train['age'] < 32)
condition3 = (train['age'] >= 32) & (train['age'] < 40)
condition4 = (train['age'] >= 40) & (train['age'] < 60)
condition5 = (train['age'] >= 60)

In [75]:
train['age_level'] = np.where(condition1, 0, np.where(condition2,1,np.where(condition3,2,np.where(condition4,3,np.where(condition5,4,"NaN")))))

In [76]:
condition1 = (test['age'] < 28)
condition2 = (test['age'] >= 28) & (test['age'] < 32)
condition3 = (test['age'] >= 32) & (test['age'] < 40)
condition4 = (test['age'] >= 40) & (test['age'] < 60)
condition5 = (test['age'] >= 60)

In [77]:
test['age_level'] = np.where(condition1, 0, np.where(condition2,1,np.where(condition3,2,np.where(condition4,3,np.where(condition5,4,"NaN")))))

In [78]:
train.drop(['age'],axis= 1, inplace=True)
test.drop(['age'],axis= 1, inplace=True)

## over_desired 파생변수 생성

희망 대출 금액과 대출 한도 금액을 비교하여 desired_amount(희망 대출 금액)보다 loan_limit(대출 한도 금액)가 클 경우 1, 그렇지 않을 경우 0의 값을 할당

In [79]:
condition_tr =((train['desired_amount'] - train['loan_limit']) >=0)
condition_te =((test['desired_amount'] - test['loan_limit']) >=0)

In [80]:
train['over_desired']=np.where(condition_tr,0,1)
test['over_desired']=np.where(condition_te,0,1)

## train_preprocessing.csv, test_preprocessing.csv 생성

In [93]:
train.shape

(6168569, 27)

In [94]:
test.shape

(3255194, 26)

In [105]:
# submission.shape

(3255194, 2)

추후 원활한 모델링을 위한 전처리 과정을 거친 train과 test의 csv 생성

In [95]:
# train.to_csv("train_preprocessing.csv")

In [96]:
# test.to_csv("test_preprocessing.csv")